# Compare pre-trained CLIP models for text-image retrieval

## Install required packages

In [ ]:
!pip install -r requirements.txt

## CLIP model

There are multiple CLIP model variations:

In [ ]:
import clip

clip.available_models()

To limit run-time of this notebook, do not use all models - modify the set:

In [ ]:
# Limit number of models to test - here, first two models
use_models = clip.available_models()[0:2]

Each model has an embedding size, needed in the text-image search application schema:

In [ ]:
embedding_info = {name: clip.load(name)[0].visual.output_dim for name in use_models}
embedding_info

## Create and deploy a text-image search app

### Create the Vespa application package

The function `create_text_image_app` below uses [the Vespa python API](https://pyvespa.readthedocs.io/en/latest/) to create an application package with fields to store each of the different types of image embedding associated with the CLIP models. It also declares the types of the text embeddings that we are going to send along with the query when searching for images, and creates one ranking profile for each (text, image) embedding model:

In [ ]:
from embedding import create_text_image_app

app_package = create_text_image_app(embedding_info)

Inspect the `schema` of the resulting application package:

In [ ]:
print(app_package.schema.schema_to_text)

### Deploy

In [ ]:
import os
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=app_package)

## Compute and feed image embeddings

Get a sample data set. See [download_flickr8k.sh](https://github.com/vespa-engine/sample-apps/blob/master/text-image-search/src/sh/download_flickr8k.sh) for how to download images. Set location of images:

In [ ]:
%env IMG_DIR=./data/Flicker8k_Dataset

For each of the CLIP models, compute the image embeddings and send it to the Vespa app:

In [ ]:
from embedding import compute_and_send_image_embeddings

compute_and_send_image_embeddings(app=app, batch_size=128, clip_model_names=use_models)

## Define QueryModel's to be evaluated

Create one `QueryModel` for each of the CLIP models. In order to do that, we need to have a function that takes a query as input and outputs the body function of a Vespa query request - example:

In [ ]:
from embedding import create_vespa_query_body_function

vespa_query_body_function = create_vespa_query_body_function("RN50")
vespa_query_body_function("this is a test query")["yql"]

With a method to create Vespa query body functions, we can create `QueryModel`s that will be used to evaluate each search configuration that is to be tested. In this case, each query model will represent a CLIP model text-image representation:

In [ ]:
from learntorank.query import QueryModel

query_models = [QueryModel(
    name=model_name, 
    body_function=create_vespa_query_body_function(model_name)
) for model_name in use_models]

A query model contains all the information that is necessary to define how the search app will match and rank documents. Use it to query the application:

In [ ]:
from embedding import plot_images
from learntorank.query import send_query

query_result = send_query(app, query="a person surfing", query_model=query_models[-1], hits = 4)

To inspect the results, use `query_result.hits[0]`. Display top two:

In [ ]:
from IPython.display import Image, display

image_file_names = [ hit["fields"]["image_file_name"] for hit in query_result.hits[:2] ]

for image in image_file_names:
    display(Image(filename=os.path.join(os.environ["IMG_DIR"], image)))

## Evaluate

Now that there is one QueryModel for each CLIP model available, it is posible to evaluate and compare them. 

Define search evaluation metrics:

In [ ]:
from learntorank.evaluation import MatchRatio, Recall, ReciprocalRank

eval_metrics = [
    MatchRatio(), # Match ratio is just to show the % of documents that are matched by ANN
    Recall(at=100), 
    ReciprocalRank(at=100)
]

Load labeled data. It was assumed that a (caption, image) pair is relevant if all three experts agreed that the caption accurately described the image:

In [ ]:
from pandas import read_csv

labeled_data = read_csv("https://data.vespa.oath.cloud/blog/flickr8k/labeled_data.csv", sep = "\t")
labeled_data.head()

Evaluate the application and return per query results:

In [ ]:
from learntorank.evaluation import evaluate

result = evaluate(
    app=app,
    labeled_data=labeled_data, 
    eval_metrics=eval_metrics, 
    query_model=query_models, 
    id_field="image_file_name",
    per_query=True
)
result.head()

Visualize RR@100:

In [ ]:
plt.plot(result.reciprocal_rank_100)
plt.ylabel("reciprocal_rank_100")
plt.show()

Compute mean and median across models:

In [ ]:
result[["model", "reciprocal_rank_100"]].groupby(
    "model"
).agg(
    Mean=('reciprocal_rank_100', 'mean'), 
    Median=('reciprocal_rank_100', 'median')
)

## Cleanup

Stop and remove the Docker container:

In [ ]:
vespa_docker.container.stop()
vespa_docker.container.remove()